In [1]:
from PIL import Image
import matplotlib.pyplot as plt
import torch
import numpy as np

from torchvision import models
from torchvision import transforms

from captum.attr import visualization as viz
from captum.attr import LayerGradCam, FeatureAblation, LayerActivation, LayerAttribution

import os
import random

In [3]:
# Default device plus free memory
torch.cuda.empty_cache()
device = "cuda"

In [4]:
# Model
fcn_model = models.segmentation.deeplabv3_mobilenet_v3_large(pretrained=True).to(device).eval()

# Input preprocessing transformation
preprocessing = transforms.Compose([transforms.Resize(640), 
                                    transforms.ToTensor()])
normalize = transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])

/home/fonta42/anaconda3/envs/torchtrainer/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/fonta42/anaconda3/envs/torchtrainer/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_MobileNet_V3_Large_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
# Get a image from the dataset
!wget -nv --directory-prefix=img/segmentation/ https://farm8.staticflickr.com/7301/8862358875_eecba9fb10_z.jpg

2023-09-08 14:26:20 URL:https://farm8.staticflickr.com/7301/8862358875_eecba9fb10_z.jpg [110427] -> "img/segmentation/8862358875_eecba9fb10_z.jpg.20" [1]


In [6]:
img = Image.open("img/segmentation/8862358875_eecba9fb10_z.jpg")
preproc_img = preprocessing(img)
preproc_img.shape

torch.Size([3, 640, 966])

In [7]:
# Normalize image and compute segmentation output
normalized_inp = normalize(preproc_img).unsqueeze(0).to(device)
normalized_inp.requires_grad = True

In [8]:
def wrapper_pix(inp, p_height = 263, p_width = 705):
    model_out = fcn_model(inp)['out']
    return (model_out[0:1, :, p_height, p_width])

In [9]:
# Gradcam for layers
layer_gradcam = LayerGradCam(wrapper_pix, fcn_model.backbone['1'].block[0][0])

In [10]:
classes = [
    "background",
    "aeroplane",
    "bicycle",
    "bird",
    "boat",
    "bottle",
    "bus",
    "car",
    "cat",
    "chair",
    "cow",
    "dining table",
    "dog",
    "horse",
    "motorbike",
    "person",
    "potted plant",
    "sheep",
    "sofa",
    "train",
    "tv_monitor"
]

In [11]:
for j in range(1, 21):
  gradcam_attr = layer_gradcam.attribute(normalized_inp, 
                                    target=j, 
                                    attr_dim_summation=False).detach().cpu()

  avrg_array = gradcam_attr.abs().mean(dim=(2,3))[0]
  idx = torch.argsort(avrg_array, descending=True)

  plt.figure(figsize=[25, 25])
  for i in range(16):
    plt.subplot(5,5, i + 1)
    _ = plt.imshow(gradcam_attr[0, idx[i]].numpy(), cmap='RdBu')
  
  _ = plt.savefig(f'./GradCamMaps/ByClass/{classes[j]}_gradcam_map.png')
  plt.close()
  
  # Default device plus free memory
  torch.cuda.empty_cache()

/tmp/ipykernel_10052/4104997671.py:14: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "facecolor" which is no longer supported as of 3.3 and will become an error two minor releases later
  _ = plt.savefig(f'./GradCamMaps/ByClass/{classes[j]}_gradcam_map.png')
/tmp/ipykernel_10052/4104997671.py:14: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "edgecolor" which is no longer supported as of 3.3 and will become an error two minor releases later
  _ = plt.savefig(f'./GradCamMaps/ByClass/{classes[j]}_gradcam_map.png')
/tmp/ipykernel_10052/4104997671.py:14: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "orientation" which is no longer supported as of 3.3 and will become an error two minor releases later
  _ = plt.savefig(f'./GradCamMaps/ByClass/{classes[j]}_gradcam_map.png')
/tmp/ipykernel_10052/4104997671.py:14: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "bbox_inches_restore" which is n

In [12]:
def generate_random_pixels(patch_width, patch_height, random_pixels):
    # Image dimensions
    height = 966
    width = 640

    # Calculate the number of patches along width and height
    num_patches_width = width // patch_width
    num_patches_height = height // patch_height

    # Iterate through the patches
    for i in range(num_patches_height):
        for j in range(num_patches_width):
            # Define the coordinates of the patch (left, upper, right, lower)
            left = j * patch_width
            upper = i * patch_height
            right = (j + 1) * patch_width
            lower = (i + 1) * patch_height

            # Generate random x and y coordinates within the patch
            x = random.randint(left, right - 1)
            y = random.randint(upper, lower - 1)

            # Add the pixel value to the list
            random_pixels.append((x, y))

In [13]:
def wrapper_pix(inp):
      model_out = fcn_model(inp)['out']
      return (model_out[0:1, :, 400, 400])
    
# Gradcam for layers
layer_gradcam = LayerGradCam(wrapper_pix, fcn_model.backbone['1'].block[0][0])


gradcam_attr = layer_gradcam.attribute(normalized_inp, 
                                  target=j, 
                                  attr_dim_summation=False).detach().cpu()

In [16]:
random_pixels = []
generate_random_pixels(100, 100, random_pixels)
for k in range(len(random_pixels)):
  def wrapper_pix(inp):
      model_out = fcn_model(inp)['out']
      return (model_out[0:1, :, random_pixels[k][0], random_pixels[k][1]])
    
  # Gradcam for layers
  layer_gradcam = LayerGradCam(wrapper_pix, fcn_model.backbone['1'].block[0][0])


  gradcam_attr = layer_gradcam.attribute(normalized_inp, 
                                    target=j, 
                                    attr_dim_summation=False).detach().cpu()

  avrg_array = gradcam_attr.abs().mean(dim=(2,3))[0]
  idx = torch.argsort(avrg_array, descending=True)

  plt.figure(figsize=[25, 25])
  for i in range(9):
    plt.subplot(3,3, i + 1)
    _ = plt.imshow(gradcam_attr[0, idx[i]].numpy(), cmap='RdBu')
  
  _ = plt.savefig(f'./GradCamMaps/ByPixels/x_{random_pixels[k][1]}_y_{random_pixels[k][0]}_gradcam_map.png')
  plt.close()
  
  # Default device plus free memory
  torch.cuda.empty_cache()

/tmp/ipykernel_8370/1177441920.py:24: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "facecolor" which is no longer supported as of 3.3 and will become an error two minor releases later
  _ = plt.savefig(f'./GradCamMaps/ByPixels/x_{random_pixels[k][1]}_y_{random_pixels[k][0]}_gradcam_map.png')
/tmp/ipykernel_8370/1177441920.py:24: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "edgecolor" which is no longer supported as of 3.3 and will become an error two minor releases later
  _ = plt.savefig(f'./GradCamMaps/ByPixels/x_{random_pixels[k][1]}_y_{random_pixels[k][0]}_gradcam_map.png')
/tmp/ipykernel_8370/1177441920.py:24: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "orientation" which is no longer supported as of 3.3 and will become an error two minor releases later
  _ = plt.savefig(f'./GradCamMaps/ByPixels/x_{random_pixels[k][1]}_y_{random_pixels[k][0]}_gradcam_map.png')
/tmp/ipykernel_8370/1177441920.py:24: 

In [ ]:
random_pixels = []
generate_random_pixels(200, 200, random_pixels)
for k in range(len(random_pixels)):
  def wrapper_pix(inp):
      model_out = fcn_model(inp)['out']
      return (model_out[0:1, :, random_pixels[k][0], random_pixels[k][1]])
    
  # Gradcam for layers
  layer_gradcam = LayerGradCam(wrapper_pix, fcn_model.backbone['1'].block[0][0])

  for j in range(1, 21):
    gradcam_attr = layer_gradcam.attribute(normalized_inp, 
                                           target=j, 
                                           attr_dim_summation=False).detach().cpu()

    avrg_array = gradcam_attr.abs().mean(dim=(2,3))[0]
    idx = torch.argsort(avrg_array, descending=True)

    plt.figure(figsize=[25, 25])
    for i in range(16):
      ax = plt.subplot(4,4, i + 1)
      ax.imshow(gradcam_attr[0, idx[i]].numpy(), cmap='RdBu')

    os.makedirs(f'./GradCamMaps/ByClassesPixels/{classes[j]}', exist_ok=True)
    _ = plt.savefig(f'./GradCamMaps/ByClassesPixels/{classes[j]}/x_{random_pixels[k][1]}_y_{random_pixels[k][0]}_gradcam_map.png')
    plt.close()
    
    # Default device plus free memory
    torch.cuda.empty_cache()

/tmp/ipykernel_6036/1764326342.py:25: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "facecolor" which is no longer supported as of 3.3 and will become an error two minor releases later
  _ = plt.savefig(f'./GradCamMaps/ByClassesPixels/{classes[j]}/x_{random_pixels[k][1]}_y_{random_pixels[k][0]}_gradcam_map.png')
/tmp/ipykernel_6036/1764326342.py:25: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "edgecolor" which is no longer supported as of 3.3 and will become an error two minor releases later
  _ = plt.savefig(f'./GradCamMaps/ByClassesPixels/{classes[j]}/x_{random_pixels[k][1]}_y_{random_pixels[k][0]}_gradcam_map.png')
/tmp/ipykernel_6036/1764326342.py:25: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "orientation" which is no longer supported as of 3.3 and will become an error two minor releases later
  _ = plt.savefig(f'./GradCamMaps/ByClassesPixels/{classes[j]}/x_{random_pixels[k][1]}_y_{random_pixels[k][

: 

: 

In [14]:
random_pixels = []
generate_random_pixels(200, 200, random_pixels)
for k in range(len(random_pixels)):
  def wrapper_pix(inp):
      model_out = fcn_model(inp)['out']
      return (model_out[0:1, :, random_pixels[k][0], random_pixels[k][1]])
    
  # Gradcam for layers
  layer_gradcam = LayerGradCam(wrapper_pix, fcn_model.backbone['1'].block[0][0])

  #plt.figure(figsize=[25, 25])
  maps = []
  for j in range(1, 21):
    gradcam_attr = layer_gradcam.attribute(normalized_inp, 
                                           target=j, 
                                           attr_dim_summation=False).detach().cpu()

    avrg_array = gradcam_attr.abs().mean(dim=(2,3))[0]
    idx = torch.argsort(avrg_array, descending=True)

    for i in range(5):
      maps.append(gradcam_attr[0, idx[i]].numpy())
  
  plt.figure(figsize=[30, 60])
  for z in range(100):
    ax = plt.subplot(20, 5, z + 1)
    ax.grid(False)
    ax.imshow(maps[z], cmap='RdBu')
    #TODO: plt.plot then scatter with coordinates
    ax.set_title(f'{classes[z // 5 + 1]}')

  plt.tight_layout()
  _ = plt.savefig(f'./GradCamMaps/ByPixelsClasses/x_{random_pixels[k][1]}_y_{random_pixels[k][0]}_gradcam_map.png')
  plt.close()

  # Default device plus free memory
  torch.cuda.empty_cache()

/tmp/ipykernel_10052/1826561659.py:33: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "facecolor" which is no longer supported as of 3.3 and will become an error two minor releases later
  _ = plt.savefig(f'./GradCamMaps/ByPixelsClasses/x_{random_pixels[k][1]}_y_{random_pixels[k][0]}_gradcam_map.png')
/tmp/ipykernel_10052/1826561659.py:33: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "edgecolor" which is no longer supported as of 3.3 and will become an error two minor releases later
  _ = plt.savefig(f'./GradCamMaps/ByPixelsClasses/x_{random_pixels[k][1]}_y_{random_pixels[k][0]}_gradcam_map.png')
/tmp/ipykernel_10052/1826561659.py:33: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "orientation" which is no longer supported as of 3.3 and will become an error two minor releases later
  _ = plt.savefig(f'./GradCamMaps/ByPixelsClasses/x_{random_pixels[k][1]}_y_{random_pixels[k][0]}_gradcam_map.png')
/tmp/ipykernel